In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import (
    Button,
    FloatSlider,
    IntSlider,
    VBox,
    HBox,
    Output,
    Layout,
    HTML,
    ToggleButton,
    Label,
)

# ---------------------------- Data generation ----------------------------- #

def generate_data(n_points: int = 300):
    """Draw x in [-10,10] and label deterministically (1 if x≥0 else 0)."""
    global X, y, X_b
    X = 20 * np.random.rand(n_points, 1) - 10
    y = (X >= 0).astype(float)
    X_b = np.c_[np.ones((n_points, 1)), X]


generate_data()

# ---------------------------- Globals & widgets --------------------------- #

x_min, x_max = -10, 10
z_ylim = (-12, 12)
output = Output()
step_info = HTML()
step_counter = 0

theta = np.zeros((2, 1))

title_html = HTML(
    """<h2>Logistic Regression: Is the number positive or negative?</h2>"""
)

# Buttons & sliders
step_button = Button(description="Step", layout=Layout(width="120px"), button_style="primary")
reset_button = Button(description="Reset Weights", layout=Layout(width="120px"))
refresh_button = Button(description="Refresh Data", layout=Layout(width="120px"))

lr_slider = FloatSlider(min=0.001, max=1.0, step=0.001, value=0.1, layout=Layout(width="200px"))
lr_row = HBox([Label("Learning rate:", layout=Layout(width="110px")), lr_slider])

batch_slider = IntSlider(min=1, max=len(X), step=10, value=len(X), layout=Layout(width="200px"))
batch_row = HBox([Label("Batch size:", layout=Layout(width="110px")), batch_slider])

random_toggle = ToggleButton(value=False, description="OFF", layout=Layout(width="60px"))
random_row = HBox([Label("Random init:", layout=Layout(width="110px")), random_toggle])

# ---------------------------- Helper functions ---------------------------- #

def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def bce_loss(logits, labels):
    """Binary cross-entropy loss on full dataset."""
    probs = sigmoid(logits)
    eps = 1e-8
    return -np.mean(labels * np.log(probs + eps) + (1 - labels) * np.log(1 - probs + eps))


def plot_state():
    """Redraw both panels with current θ."""
    with output:
        output.clear_output(wait=True)
        fig, (ax_z, ax_p) = plt.subplots(
            2,
            1,
            sharex=True,
            figsize=(9, 8),
            gridspec_kw={"height_ratios": [1, 1]},
        )

        xs = np.linspace(x_min, x_max, 500)
        z_line = theta[0] + theta[1] * xs
        p_line = sigmoid(z_line)

        # Linear score
        ax_z.plot(xs, z_line, "k-", label="z = Intercept + Slope·x")
        ax_z.axhline(0, color="grey", lw=0.8, ls="--")
        ax_z.axvline(0, color="grey", lw=0.8, ls=":")
        ax_z.set_ylabel("Linear score z")
        ax_z.set_ylim(*z_ylim)
        ax_z.legend(loc="upper left")

        # Sigmoid panel
        ax_p.scatter(X, y, c=y.ravel(), cmap="coolwarm", alpha=0.7, label="data")
        ax_p.plot(xs, p_line, "k-", label="σ(z)")
        ax_p.axhline(0.5, color="grey", lw=0.8, ls="--")
        ax_p.axvline(0, color="grey", lw=0.8, ls=":")
        ax_p.set_xlabel("x value")
        ax_p.set_ylabel("Logistic Regression Score p")
        ax_p.set_ylim(-0.05, 1.05)
        ax_p.legend(loc="upper left")

        plt.tight_layout()
        plt.show()


# ---------------------------- State management ---------------------------- #

def reset_weights():
    """Reset θ and counters."""
    global theta, step_counter
    step_counter = 0
    batch_slider.max = len(X)
    theta = np.random.randn(2, 1) if random_toggle.value else np.zeros((2, 1))
    total_loss = bce_loss(X_b @ theta, y)
    update_info(None, None, total_loss)
    plot_state()


def step_once(_=None):
    """Single gradient-descent step and refresh plots."""
    global theta, step_counter
    m = len(X_b)
    k = batch_slider.value
    idx = np.random.choice(m, k, replace=False)
    Xb = X_b[idx]
    yb = y[idx]

    old_theta = theta.copy()

    preds = sigmoid(Xb @ theta)
    grads = (1 / k) * Xb.T @ (preds - yb)
    theta -= lr_slider.value * grads

    step_counter += 1
    total_loss = bce_loss(X_b @ theta, y)
    update_info(old_theta, grads, total_loss)
    plot_state()


# ---------------------------- Info panel text ----------------------------- #

def bold(text):
    return f"<b>{text}</b>"


def update_info(old, grad, loss):
    if old is None:
        step_info.value = (
            f"{bold('Step:')} 0 | {bold('Intercept (w₀):')} {theta[0,0]:.4f}, "
            f"{bold('Slope (w₁):')} {theta[1,0]:.4f}<br>"
            f"{bold('Loss (Binary Cross-Entropy):')} {loss:.4f}"
        )
    else:
        step_info.value = (
            f"{bold('Step:')} {step_counter}<br>"
            f"{bold('Old Intercept (w₀):')} {old[0,0]:.4f}, {bold('Slope (w₁):')} {old[1,0]:.4f}<br>"
            f"{bold('Intercept gradient:')} {grad[0,0]:.4f}, {bold('Slope gradient:')} {grad[1,0]:.4f}<br>"
            f"{bold('New Intercept (w₀):')} {theta[0,0]:.4f}, {bold('Slope (w₁):')} {theta[1,0]:.4f}<br>"
            f"{bold('Loss (BCE):')} {loss:.4f}"
        )


# ---------------------------- Event wiring ------------------------------- #

step_button.on_click(step_once)
reset_button.on_click(lambda _: reset_weights())
refresh_button.on_click(lambda _: (generate_data(), reset_weights()))
random_toggle.observe(
    lambda c: (
        setattr(random_toggle, "description", "ON" if c.new else "OFF"),
        reset_weights(),
    ),
    names="value",
)

# ---------------------------- Layout ------------------------------------ #

controls = VBox(
    [
        HBox([step_button]),
        HBox([reset_button, refresh_button], layout=Layout(gap="10px")),
        lr_row,
        batch_row,
        random_row,
    ],
    layout=Layout(gap="10px"),
)

top = HBox([controls, step_info], layout=Layout(gap="40px"))
ui = VBox([title_html, top, output], layout=Layout(padding="10px"))

# ---------------------------- Kick-off ----------------------------------- #

reset_weights()
ui